In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# NLP Complaint Intelligence System  
## Phase 1: Data Understanding & Column Strategy (Industry Grade)

### Why this phase exists

In real-world machine learning projects, the first and most critical step is **not modeling**.  
It is understanding **what data represents the business problem** and **which signals should drive decisions**.

Poor column selection leads to:
- Noisy models
- Overfitting
- Irrelevant predictions
- Low business trust

This phase ensures that:
- Every column used has a clear purpose
- Every column ignored is ignored intentionally
- The system design reflects real company workflows

---

## Step 1: Load Raw Dataset from Storage

### Why this step is necessary

Production systems never assume local files.
Data is always loaded from a controlled, external source to ensure:
- Reproducibility
- Data lineage
- Environment independence

This mirrors how data is accessed in enterprise ML pipelines.




## Step 2: Inspect Dataset Structure

### Why this step is required

Before selecting or cleaning any column, we must understand:
- How many records exist
- What columns are available
- Whether the schema matches expectations

This prevents wrong assumptions early in the pipeline.


## Large-Scale Dataset Reduction Using Chunk-Based Streaming

### Problem Context

The raw CFPB complaints dataset contains **12.8 million+ rows and 18 columns**.  
Loading or copying this dataset directly into memory causes:

- Memory crashes (`MemoryError`)
- Extremely slow experimentation
- Unusable local and Colab environments

In real-world machine learning systems, **datasets of this size are never fully loaded into memory**.

---

## Why This Method Is Used

This approach uses **chunk-based streaming** to process large datasets safely and efficiently.

Instead of loading the entire dataset at once, the file is read in **small, fixed-size chunks**, processed independently, and written directly to disk.

This mirrors how **enterprise ML pipelines** handle large-scale data.

---

## Key Design Decisions Explained

### 1. Column-Level Loading (`usecols`)

Only the **three required columns** are loaded:
- `Date received`
- `Consumer complaint narrative`
- `Product`

This avoids wasting memory on irrelevant fields and reduces I/O overhead.

---

### 2. Chunk-Based Reading (`chunksize`)

The dataset is read in chunks of **200,000 rows**:
- Prevents RAM exhaustion
- Keeps memory usage stable
- Allows processing datasets with tens of millions of rows

This is the standard technique used in production data engineering workflows.

---

### 3. Temporal Filtering (Year ≥ 2021)

Only recent complaints are retained:
- Captures **modern customer language**
- Reflects **current products and policies**
- Improves real-world deployment relevance

Older data is intentionally excluded to avoid outdated patterns.

---

### 4. Incremental Random Sampling

Each chunk is randomly sampled and appended until the target size is reached:
- Ensures diversity across time and products
- Avoids loading all data at once
- Keeps total dataset size bounded (≈ 400,000 rows)

This balances **representativeness and computational efficiency**.

---

### 5. Disk-Based Appending (Not In-Memory Aggregation)

Processed data is written directly to disk:
- No large intermediate DataFrames
- No `copy()` operations on massive objects
- No memory spikes

This design ensures the pipeline completes reliably even on limited hardware.

---

## Why This Is Industry-Grade

This approach demonstrates:
- Awareness of real memory constraints
- Scalable data handling skills
- Production-oriented thinking
- Separation of data engineering and modeling concerns

This is **how large NLP datasets are prepared in real companies**, not how toy projects are built.

---

## Final Outcome

- Original dataset size: **12.8M+ rows**
- Final modeling dataset: **~400K rows**
- Memory usage: **stable**
- Runtime: **10–20 minutes**
- Ready for embeddings, modeling, and deployment

This reduced dataset preserves signal quality while enabling fast, reliable experimentation.


In [3]:
import pandas as pd
import os

input_path = r"/content/drive/MyDrive/PROJECTS/EXTRA PROJECT/NLP_PROJECT_COMPLAINT/data/complaints.csv"
output_path = "/content/drive/MyDrive/PROJECTS/EXTRA PROJECT/NLP_PROJECT_COMPLAINT/data/short_cleaned_complaints.csv"

use_cols = [
    "Date received",
    "Consumer complaint narrative",
    "Product"
]

chunk_size = 200_000
target_rows = 400_000

if os.path.exists(output_path):
    os.remove(output_path)

written_rows = 0

for chunk in pd.read_csv(
    input_path,
    usecols=use_cols,
    chunksize=chunk_size,
    engine="python",
    on_bad_lines="skip"
):
    chunk["Date received"] = pd.to_datetime(
        chunk["Date received"], errors="coerce"
    )

    chunk = chunk[chunk["Date received"].dt.year >= 2021]

    chunk = chunk.dropna(
        subset=["Consumer complaint narrative", "Product"]
    )

    if len(chunk) == 0:
        continue

    remaining = target_rows - written_rows
    if remaining <= 0:
        break

    sample_n = min(len(chunk), remaining)
    chunk = chunk.sample(n=sample_n, random_state=42)

    chunk.to_csv(
        output_path,
        mode="a",
        header=not os.path.exists(output_path),
        index=False
    )

    written_rows += len(chunk)
    print(f"Saved rows so far: {written_rows}")

    if written_rows >= target_rows:
        print("Target reached. Stopping further processing.")
        break

print("Final dataset saved successfully.")
print("Total rows:", written_rows)


Saved rows so far: 22126
Saved rows so far: 49545
Saved rows so far: 82919
Saved rows so far: 119463
Saved rows so far: 160752
Saved rows so far: 205754
Saved rows so far: 256665
Saved rows so far: 310383
Saved rows so far: 365714
Saved rows so far: 400000
Target reached. Stopping further processing.
Final dataset saved successfully.
Total rows: 400000


In [4]:
df = pd.read_csv(output_path)
df.shape

(400000, 3)

In [5]:
df.shape

(400000, 3)

In [6]:
df.columns

Index(['Date received', 'Product', 'Consumer complaint narrative'], dtype='object')

In [7]:
df.head()

,Date received,Product,Consumer complaint narrative
0,2025-07-30,Credit reporting or other personal consumer re...,I am writing to formally submit a complaint un...
1,2024-04-24,Credit reporting or other personal consumer re...,15UCS 1681 section 602 A. States I have the ri...
2,2025-08-29,Credit reporting or other personal consumer re...,I am writing to have the following information...
3,2025-10-27,Credit reporting or other personal consumer re...,Transunion still has a bankruptcy on my accoun...
4,2023-04-21,"Credit reporting, credit repair services, or o...",This account was opened over 10 years ago. Thi...


## Step 3: Select Columns Relevant for Complaint Intelligence

### Why this step is required

In industry, using every column is considered poor design.
Only columns that directly support the business objective
should be part of the modeling pipeline.

For complaint intelligence, we need:
- Complaint text as input
- Product category as target label


In [8]:
df = df[["Consumer complaint narrative", "Product"]]

## Step 4: Remove Records Without Complaint Text

### Why this step is required

Text-based models cannot learn from missing or empty text.
Rows without complaint narratives add no semantic value
and can bias dataset statistics.

Industry practice is to drop such rows explicitly.


In [9]:
df = df.dropna(subset=["Consumer complaint narrative"])

## Step 5: Rename Columns for Pipeline Consistency

### Why this step is required

Clean and consistent column names:
- Improve readability
- Reduce code complexity
- Match production ML conventions

This makes the pipeline easier to maintain and explain.


In [10]:
df = df.rename(columns={
    "Consumer complaint narrative": "text",
    "Product": "label"
})

## Step 6: Normalize Text Case

### Why this step is required

Machine learning models treat different casing as different tokens.
Lowercasing removes meaningless variation and improves
model generalization.

This is a universal NLP best practice.


In [11]:
df["text"] = df["text"].str.lower()

## Step 7: Remove Non-Linguistic Noise from Text

### Why this step is required

Customer complaints often include:
- URLs
- Email addresses
- Numbers
- Special characters

These elements do not add semantic meaning and degrade
embedding quality. Removing them improves signal clarity.

In [12]:
import re

def clean_text(text):
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"\S+@\S+", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["text"] = df["text"].apply(clean_text)

## Step 8: Remove Very Short Complaints

### Why this step is required

Extremely short complaints lack sufficient context.
They provide little learning signal and increase ambiguity.

Industry pipelines enforce a minimum text length
to ensure meaningful training data.


In [13]:
df = df[df["text"].str.split().str.len() >= 5]

## Step 9: Reset Index After Cleaning

### Why this step is required

After dropping rows, index values become inconsistent.
Resetting the index ensures clean dataset structure
for downstream processing.


In [14]:
df = df.reset_index(drop=True)

## Step 10: Final Data Quality Check

### Why this step is required

Before saving the cleaned dataset, we must confirm:
- No missing values remain
- Text looks clean and meaningful
- Dataset is ready for modeling


In [15]:
df.head()

,text,label
0,i am writing to formally submit a complaint un...,Credit reporting or other personal consumer re...
1,ucs section a states i have the right to priva...,Credit reporting or other personal consumer re...
2,i am writing to have the following information...,Credit reporting or other personal consumer re...
3,transunion still has a bankruptcy on my accoun...,Credit reporting or other personal consumer re...
4,this account was opened over years ago this da...,"Credit reporting, credit repair services, or o..."


In [16]:
df.isnull().sum()

,0
text,0
label,0


In [17]:
df.shape

(399811, 2)

## Step 11: Save Cleaned Dataset for Modeling

### Why this step is required

Cleaned datasets are treated as reusable assets.
They are stored separately from raw data and reused
across embeddings, models, and experiments.

This file becomes the trusted input for all future phases.


In [18]:
clean_path = "/content/drive/MyDrive/PROJECTS/EXTRA PROJECT/NLP_PROJECT_COMPLAINT/data/complaints_clean.csv"
df.to_csv(clean_path, index=False)

## PHASE 1 Completion Summary

At the end of this phase:
- Raw complaint data has been converted into clean text
- Noise and inconsistencies have been removed
- Column selection reflects business intent
- A production-ready dataset has been created

This demonstrates real-world NLP pipeline design.
